# Pytree 사용하기

JAX는 딕셔너리, 배열, 중첩 리스트, 딕셔너리 리스트 혼합 중첩 등 다양한 구조에 대한 연산을 지원하는데, 이러한 구조들을 JAX에서는 pytree라고 부릅니다. 해당 문서에서는 pytree를 잘 사용하는 법과 실수할 수 있는 부분들을 짚고 넘어가겠습니다.

## pytree란

[공식 도큐먼트](https://jax.readthedocs.io/en/latest/working-with-pytrees.html) 에서 pytree에 대해 설명한 글을 직역하면 다음과 같습니다(어떻게 더 잘 번역을 해야할지 감이 안와 공식 도큐먼트를 그대로 인용합니다.):

*파이 트리는 컨테이너 형태의 Python 객체들로 구성된 구조입니다. “리프(leaf)” 파이 트리와 더 많은 파이 트리들로 이루어질 수 있습니다. 파이 트리는 리스트, 튜플, 사전을 포함할 수 있습니다. 리프(leaf)는 배열과 같이 파이 트리가 아닌 모든 것을 의미하지만, 단일 리프도 파이 트리입니다.*

머신러닝의 관점에서 pytree는 다음과 같은 정보를 포함할 수 있습니다:

- 모델 파라메터
- 데이터셋
- 강화학습 문제에서의 에이전트 observation

아래는 간단한 pytree 예제입니다. example_trees 라는 pytree object를 만들어 보겠습니다:

In [2]:
import jax
import jax.numpy as jnp

example_trees = [
    [1, 'a', object()],
    (1, (2, 3), ()),
    [1, {'k1': 2, 'k2': (3, 4)}, 5],
    {'a': 2, 'b': (2, 3)},
    jnp.array([1, 2, 3]),
]

`jax.tree.leaves()`는 리프 노드들을 flatten한 값을 추출합니다. 아래는 `jax.tree.leaves()` 없이 출력한 결과입니다:

In [3]:
for pytree in example_trees:
    print(f"{repr(pytree):<45} has {len(pytree)} leaves: {pytree} leavers: {pytree}")

[1, 'a', <object object at 0x112018480>]      has 3 leaves: [1, 'a', <object object at 0x112018480>] leavers: [1, 'a', <object object at 0x112018480>]
(1, (2, 3), ())                               has 3 leaves: (1, (2, 3), ()) leavers: (1, (2, 3), ())
[1, {'k1': 2, 'k2': (3, 4)}, 5]               has 3 leaves: [1, {'k1': 2, 'k2': (3, 4)}, 5] leavers: [1, {'k1': 2, 'k2': (3, 4)}, 5]
{'a': 2, 'b': (2, 3)}                         has 2 leaves: {'a': 2, 'b': (2, 3)} leavers: {'a': 2, 'b': (2, 3)}
Array([1, 2, 3], dtype=int32)                 has 3 leaves: [1 2 3] leavers: [1 2 3]


아래 코드는 `jax.tree.leaves()`를 사용하여 리프 노드를 출력한 결과입니다:

In [4]:
for pytree in example_trees:
    leaves = jax.tree.leaves(pytree)
    print(f"{repr(pytree):<45} has {len(leaves)} leaves: {leaves} leavers: {leaves}")

[1, 'a', <object object at 0x112018480>]      has 3 leaves: [1, 'a', <object object at 0x112018480>] leavers: [1, 'a', <object object at 0x112018480>]
(1, (2, 3), ())                               has 3 leaves: [1, 2, 3] leavers: [1, 2, 3]
[1, {'k1': 2, 'k2': (3, 4)}, 5]               has 5 leaves: [1, 2, 3, 4, 5] leavers: [1, 2, 3, 4, 5]
{'a': 2, 'b': (2, 3)}                         has 3 leaves: [2, 2, 3] leavers: [2, 2, 3]
Array([1, 2, 3], dtype=int32)                 has 1 leaves: [Array([1, 2, 3], dtype=int32)] leavers: [Array([1, 2, 3], dtype=int32)]


파이썬 컨테이너 형태의 트리 구조로 이루어진 객체들은 모두 pytree로 취급할 수 있습니다. JAX는 pytree 레지스트리를 가지고 있고 레지스트리에 등록된 객체를 pytree로 인식하는데, 리스트, 딕셔너리, 튜플은 기본적으로 pytree 레지스트리에 등록되어 있습니다. 클래스 형태 또한 pytree 레지스트리에 등록한다면 pytree로 인식되게 할 수 있으며, 레지스트리에 등록되지 않은 모든 객체들은 리프 노드로 취급됩니다.

아래 [사용자 정의 pytree](#사용자-정의-pytree) 섹션에서 커스텀 pytree를 만들고 pytree 레지스트리에 등록하는 방법을 알아보겠습니다.

## 대표적인 pytree 함수들

JAX에는 pytree를 다루기 위한 다양한 함수들이 있으며, `jax.tree_util` 아래에 위치하고 있습니다. 하지만 편의를 위해 `jax.tree`로도 접근할 수 있습니다(Aliasing).

### `jax.tree.map`

가장 흔하게 사용되는 pytree 함수는 `jax.tree.map()`입니다. 파이썬 기본함수 `map`과 유사하게 작동하지만 pytree 내부 모든 객체에 대해서 적용됩니다:

In [5]:
list_of_lists = [
    [1, 2, 3],
    [1, 2],
    [1, 2, 3, 4],
]

jax.tree.map(lambda x: x*2, list_of_lists)

[[2, 4, 6], [2, 4], [2, 4, 6, 8]]

In [6]:
another_list_of_lists = list_of_lists
jax.tree.map(lambda x, y: x+y, list_of_lists, another_list_of_lists)

[[2, 4, 6], [2, 4], [2, 4, 6, 8]]

여러개의 인자를 사용할 떄 연산이 이루어지는 pytree간의 형태는 같아야 합니다. 

### 예시 1: `jax.tree.map`로 딥러닝 모델 파라메터 다루기

아래 코드와 같이 간단한 신경망(MLP)을 다룰 때, 신경망의 파라메를 pytree로 정의할 수 있습니다:

In [13]:
import numpy as np

def init_mlp_params(layer_widths):
  params = []
  for n_in, n_out in zip(layer_widths[:-1], layer_widths[1:]):
    params.append(
        dict(weights=np.random.normal(size=(n_in, n_out)) * np.sqrt(2/n_in),
             biases=np.ones(shape=(n_out,))
            )
    )
  return params

params = init_mlp_params([1, 128, 128, 1])

> **NOTE**
> 위의 공식 예제 코드에서 신경망 파라메터를 NumPy를 써서 정의한 것을 볼 수 있는데, 왜 이렇게 한지는 모르겠습니다. numpy 배열을 넘겨줘도 JIT 컴파일은 잘 되는데, 제 생각에는 NumPy 특성상 JAX보다 더 직관적인 코드를 보여줄 수 있어서 가독성 측면에서 Jax.Array 대신 Numpy 배열을 사용한 것 같습니다.

In [14]:
jax.tree.map(lambda x: x.shape, params)

[{'biases': (128,), 'weights': (1, 128)},
 {'biases': (128,), 'weights': (128, 128)},
 {'biases': (1,), 'weights': (128, 1)}]

In [15]:
# Define the forward pass.
def forward(params, x):
  *hidden, last = params
  for layer in hidden:
    x = jax.nn.relu(x @ layer['weights'] + layer['biases'])
  return x @ last['weights'] + last['biases']

# Define the loss function.
def loss_fn(params, x, y):
  return jnp.mean((forward(params, x) - y) ** 2)

# Set the learning rate.
LEARNING_RATE = 0.0001

# Using the stochastic gradient descent, define the parameter update function.
# Apply `@jax.jit` for JIT compilation (speed).
@jax.jit
def update(params, x, y):
  # Calculate the gradients with `jax.grad`.
  grads = jax.grad(loss_fn)(params, x, y)
  # Note that `grads` is a pytree with the same structure as `params`.
  # `jax.grad` is one of many JAX functions that has
  # built-in support for pytrees.
  # This is useful - you can apply the SGD update using JAX pytree utilities.
  return jax.tree.map(
      lambda p, g: p - LEARNING_RATE * g, params, grads
  ), grads

## 사용자 정의 pytree

`jax.tree_util.register_pytree_node()`를 사용하여 원하는 pytree 노드를 만들 수 있습니다.

만약 별다른 처리 없이 사용자 정의 클래스를 pytree 안에 포함시킨다면, 비록 클래스 안에 pytree 객체가 포함되어 있다고 하더라도 JAX가 인식하지 못합니다. 아래와 같이 사용자 정의 클래스를 정의해 봅시다:

In [79]:
class Special(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y
jax.tree.leaves([
    Special(0, 1),
    Special(2, 4),
])

[<__main__.Special at 0x158dad990>, <__main__.Special at 0x158b6c990>]

때문에 아래와 같은 연산은 오류가 나게 됩니다. 클래스 내부 변수를 리프처럼 다루려고 하는데, JAX가 보기에는 클래스 자체가 리프가 되기 때문입니다:

In [80]:
jax.tree.map(lambda x: x + 1,
    [
        Special(0, 1),
        Special(2, 4),
    ])

TypeError: unsupported operand type(s) for +: 'Special' and 'int'

사용자 정의 클래스를 pytree 내부에서 노드로 작용하게끔 만들고 싶다면 JAX의 pytree 레지스트리에 등록을 하면 됩니다. 그럼 해당 클래스는 전역 레지스트리에 등록되게 됩니다. 레지스트리에 등록된 타입은 재귀적으로 탐색하여 찾게 됩니다.

아래와 같이 `jax.tree_util.register_pytree_node()`를 사용하여 레지스트리에 등록할 수 있습니다:

In [82]:
from jax.tree_util import register_pytree_node

class RegisteredSpecial(Special):
    def __repr__(self):
        return "RegisteredSpecial(x={}, y={})".format(self.x, self.y)
    
def special_flatten(v):
    """사용자 정의 클래스의 Flatten 연산을 정의해 줍니다.

    Args:
        v: flatten이 일어날 사용자 정의 클래스
    
    Returns:
        flatten이 적용된 iterable 객채와, 이후 unflatten이 일어날 때 필요한 정보를 담은 auxiliary를 리턴합니다.
        auxiliary는 이후 unflatten을 위해 treedef에 저장됩니다. 예를 들어 딕셔너리의 키값이 auxiliary가 될 수 있습니다. 
    """
    children = (v.x, v.y)
    aux_data = None
    return (children, aux_data)

def special_unflatten(aux_data, children):
    """사용자 정의 클래스의 Unflatten 연산을 정의해 줍니다.

    Args:
        aux_data: 시용자 클래스 객체가 flatten되면서 정의된 데이터
        children: flatten된 사용자 정의 객체
    
    Returns:
        children과 aux_data를 통해 unflatten(재구성)된 사용자 정의 객체
    """
    return RegisteredSpecial(*children)

# 전역 레지스트리에 등록
register_pytree_node(
    RegisteredSpecial,
    special_flatten,    # Jax에 children node가 뭔지 알려줌
    special_unflatten,  # Jax에 어떻게 RegisteredSpecial객체를 unflatten 시키는지 알려줌
)

이제 사용자 정의 클래스를 다음과 같이 노드로써 사용 가능합니다:

In [83]:
jax.tree.map(lambda x: x + 1,
    [
        RegisteredSpecial(0, 1),
        RegisteredSpecial(2, 4),
    ])

[RegisteredSpecial(x=1, y=2), RegisteredSpecial(x=3, y=5)]

모던 파이썬에는 사용자 정의 객체를 더욱 편리하게 만들 수 있도록 하는 편의기능이 많이 있습니다. 어떤건 JAX와 잘 작동하면서도 어떤건 주의가 필요합니다.

예를 들어 파이썬의 `NamedTuple` 서브클래스는 따로 레지스트리에 등록할 필요가 없이 pytree의 node로써 사용할 수 있습니다:

In [84]:
from typing import NamedTuple, Any

class MyOtherContainer(NamedTuple):
    name: str
    a: Any
    b: Any
    c: Any

# NamedTuple은 pytree의 Node로 인식되기 때문에
# 별다른 레지스트리 등록 없이 사용 가능
jax.tree.leaves([
    MyOtherContainer('Alice', 1, 2, 3),
    MyOtherContainer('Bob', 4, 5, 6),
])

['Alice', 1, 2, 3, 'Bob', 4, 5, 6]